### The changes
- Apply Target Encoding to the categorical columns
- Increase the data size from 3000 to 30000
- Apply Random Forest algorithm when filling in NaNs with suitable values
- SHAP with XGBoost and Permutation Importance were applied

In [25]:
!pip install category_encoders
!pip install shap
!pip install xgboost
!pip install lightgbm
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.feature_selection import mutual_info_regression, SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from category_encoders import TargetEncoder
import shap
from sklearn.inspection import permutation_importance
from google.colab import drive



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189434 entries, 0 to 1189433
Data columns (total 24 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   Year                             1189434 non-null  int64  
 1   Month                            1189434 non-null  int64  
 2   Region_Name                      1189434 non-null  object 
 3   Floor                            1189434 non-null  float64
 4   Building_Age                     1189434 non-null  int64  
 5   Building_Use                     1189434 non-null  object 
 6   JS_BA                            1189434 non-null  float64
 7   Population                       1189434 non-null  int64  
 8   IR                               1182725 non-null  float64
 9   UR                               1182725 non-null

,Year,Month,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,JS_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School
0,2011,1,관악구,12.0,6,오피스텔,25.05,521193,2.75,3.8,...,81.2,NaN,37.474500,126.981390,15800.00,12000,201101,869.157047,1510.318477,488.498007
1,2011,2,관악구,13.0,6,오피스텔,25.05,521193,2.75,4.5,...,82.0,NaN,37.474500,126.981390,16875.00,10000,201102,869.157047,1510.318477,488.498007
2,2011,2,관악구,13.0,6,오피스텔,26.18,521193,2.75,4.5,...,82.0,NaN,37.474500,126.981390,16875.00,10500,201102,869.157047,1510.318477,488.498007
3,2011,5,관악구,9.0,6,오피스텔,27.15,521193,3.00,3.2,...,80.6,NaN,37.474500,126.981390,17216.67,12000,201105,869.157047,1510.318477,488.498007
4,2011,10,관악구,6.0,6,오피스텔,27.15,521193,3.25,2.9,...,79.4,NaN,37.474500,126.981390,22000.00,11500,201110,869.157047,1510.318477,488.498007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189429,2022,11,송파구,14.0,14,아파트,84.80,652993,3.25,2.3,...,146.8,NaN,37.514069,127.081381,193312.50,96900,202211,706.916213,3320.439281,105.387151
1189430,2022,11,서초구,16.0,17,아파트,29.60,402936,3.25,2.3,...,146.8,NaN,37.486044,126.984201,33000.00,25500,202211,896.312146,1249.195643,307.492246
1189431,2022,11,강서구,7.0,6,오피스텔,25.55,567898,3.25,2.3,...,146.8,NaN,37.560579,126.837594,18125.00,15180,202211,945.993946,1247.744788,414.012705
1189432,2022,11,송파구,3.0,9,오피스텔,26.82,652993,3.25,2.3,...,146.8,NaN,37.480095,127.119684,21100.00,19800,202211,661.207119,4507.341345,715.421992


In [57]:


drive.mount('/content/drive')
EDA_data = pd.read_csv('/content/drive/MyDrive/sy_data/EDA_data(+distance).csv', encoding='utf-8')
EDA_data.info()
EDA_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189434 entries, 0 to 1189433
Data columns (total 24 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   Year                             1189434 non-null  int64  
 1   Month                            1189434 non-null  int64  
 2   Region_Name                      1189434 non-null  object 
 3   Floor                            1189434 non-null  float64
 4   Building_Age                     1189434 non-null  int64  
 5   Building_Use                     1189434 non-null  object 
 6   JS_BA                            1189434 non-null  float64
 7   Population                       1189434 non-null  int64  
 8   IR                               1182725 non-null  float64
 9   UR                               1182725 non-null

,Year,Month,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,JS_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School
0,2011,1,관악구,12.0,6,오피스텔,25.05,521193,2.75,3.8,...,81.2,NaN,37.474500,126.981390,15800.00,12000,201101,869.157047,1510.318477,488.498007
1,2011,2,관악구,13.0,6,오피스텔,25.05,521193,2.75,4.5,...,82.0,NaN,37.474500,126.981390,16875.00,10000,201102,869.157047,1510.318477,488.498007
2,2011,2,관악구,13.0,6,오피스텔,26.18,521193,2.75,4.5,...,82.0,NaN,37.474500,126.981390,16875.00,10500,201102,869.157047,1510.318477,488.498007
3,2011,5,관악구,9.0,6,오피스텔,27.15,521193,3.00,3.2,...,80.6,NaN,37.474500,126.981390,17216.67,12000,201105,869.157047,1510.318477,488.498007
4,2011,10,관악구,6.0,6,오피스텔,27.15,521193,3.25,2.9,...,79.4,NaN,37.474500,126.981390,22000.00,11500,201110,869.157047,1510.318477,488.498007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189429,2022,11,송파구,14.0,14,아파트,84.80,652993,3.25,2.3,...,146.8,NaN,37.514069,127.081381,193312.50,96900,202211,706.916213,3320.439281,105.387151
1189430,2022,11,서초구,16.0,17,아파트,29.60,402936,3.25,2.3,...,146.8,NaN,37.486044,126.984201,33000.00,25500,202211,896.312146,1249.195643,307.492246
1189431,2022,11,강서구,7.0,6,오피스텔,25.55,567898,3.25,2.3,...,146.8,NaN,37.560579,126.837594,18125.00,15180,202211,945.993946,1247.744788,414.012705
1189432,2022,11,송파구,3.0,9,오피스텔,26.82,652993,3.25,2.3,...,146.8,NaN,37.480095,127.119684,21100.00,19800,202211,661.207119,4507.341345,715.421992


In [26]:
EDA_data['YearMonth'] = pd.to_datetime(EDA_data['YearMonth'], format='%Y%m')
EDA_data


,Year,Month,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,JS_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School
0,2011,1,관악구,12.0,6,오피스텔,25.05,521193,2.75,3.8,...,81.2,NaN,37.474500,126.981390,15800.00,12000,2011-01-01,869.157047,1510.318477,488.498007
1,2011,2,관악구,13.0,6,오피스텔,25.05,521193,2.75,4.5,...,82.0,NaN,37.474500,126.981390,16875.00,10000,2011-02-01,869.157047,1510.318477,488.498007
2,2011,2,관악구,13.0,6,오피스텔,26.18,521193,2.75,4.5,...,82.0,NaN,37.474500,126.981390,16875.00,10500,2011-02-01,869.157047,1510.318477,488.498007
3,2011,5,관악구,9.0,6,오피스텔,27.15,521193,3.00,3.2,...,80.6,NaN,37.474500,126.981390,17216.67,12000,2011-05-01,869.157047,1510.318477,488.498007
4,2011,10,관악구,6.0,6,오피스텔,27.15,521193,3.25,2.9,...,79.4,NaN,37.474500,126.981390,22000.00,11500,2011-10-01,869.157047,1510.318477,488.498007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189429,2022,11,송파구,14.0,14,아파트,84.80,652993,3.25,2.3,...,146.8,NaN,37.514069,127.081381,193312.50,96900,2022-11-01,706.916213,3320.439281,105.387151
1189430,2022,11,서초구,16.0,17,아파트,29.60,402936,3.25,2.3,...,146.8,NaN,37.486044,126.984201,33000.00,25500,2022-11-01,896.312146,1249.195643,307.492246
1189431,2022,11,강서구,7.0,6,오피스텔,25.55,567898,3.25,2.3,...,146.8,NaN,37.560579,126.837594,18125.00,15180,2022-11-01,945.993946,1247.744788,414.012705
1189432,2022,11,송파구,3.0,9,오피스텔,26.82,652993,3.25,2.3,...,146.8,NaN,37.480095,127.119684,21100.00,19800,2022-11-01,661.207119,4507.341345,715.421992


In [58]:
# Sorting by the columns in descending order
EDA_data = EDA_data.sort_values(by=['YearMonth', 'Region_Name', 'Building_Use'], ascending=[True,True,True])

EDA_data

,Year,Month,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,JS_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School
315,2011,1,강남구,4.0,8,아파트,95.73,557563,2.75,3.8,...,81.2,NaN,37.502593,127.031140,54500.00,32000,201101,458.765733,1591.466670,723.715703
1018,2011,1,강남구,19.0,5,아파트,63.09,557563,2.75,3.8,...,81.2,NaN,37.485309,127.034404,38600.00,30000,201101,107.550575,1691.882920,270.721735
1020,2011,1,강남구,9.0,5,아파트,67.50,557563,2.75,3.8,...,81.2,NaN,37.485309,127.034404,38600.00,31000,201101,107.550575,1691.882920,270.721735
1055,2011,1,강남구,5.0,6,아파트,129.18,557563,2.75,3.8,...,81.2,NaN,37.506861,127.028750,61500.00,40000,201101,273.072763,1134.433254,285.120130
1058,2011,1,강남구,1.0,6,아파트,59.61,557563,2.75,3.8,...,81.2,NaN,37.498777,127.049879,85672.73,30000,201101,156.475666,2614.395870,244.731907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125397,2023,9,중랑구,10.0,27,아파트,49.77,384272,NaN,NaN,...,NaN,NaN,37.615262,127.091014,39000.00,30000,202309,531.732619,1429.809931,121.712855
1125400,2023,9,중랑구,5.0,27,아파트,39.84,384272,NaN,NaN,...,NaN,NaN,37.615262,127.091014,39000.00,24000,202309,531.732619,1429.809931,121.712855
1125416,2023,9,중랑구,3.0,26,아파트,50.37,384272,NaN,NaN,...,NaN,NaN,37.597149,127.081907,54000.00,28800,202309,255.627811,1952.815142,188.219471
1125346,2023,9,중랑구,10.0,5,연립다세대,24.93,384272,NaN,NaN,...,NaN,NaN,37.594435,127.079118,21000.00,18000,202309,296.362951,1574.826226,495.776743


In [28]:
columns_to_drop = ['Year', 'Month']
new_data = EDA_data.drop(columns = columns_to_drop)
new_data.info()
new_data



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1189434 entries, 315 to 1125360
Data columns (total 22 columns):
 #   Column                           Non-Null Count    Dtype         
---  ------                           --------------    -----         
 0   Region_Name                      1189434 non-null  object        
 1   Floor                            1189434 non-null  float64       
 2   Building_Age                     1189434 non-null  int64         
 3   Building_Use                     1189434 non-null  object        
 4   JS_BA                            1189434 non-null  float64       
 5   Population                       1189434 non-null  int64         
 6   IR                               1182725 non-null  float64       
 7   UR                               1182725 non-null  float64       
 8   LC_index                         1182725 non-null  float64       
 9   CA_index                         1182725 non-null  float64       
 10  TC_index                    

,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,LC_index,CA_index,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,JS_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School
315,강남구,4.0,8,아파트,95.73,557563,2.75,3.8,72.0,78.2,...,81.2,NaN,37.502593,127.031140,54500.00,32000,2011-01-01,458.765733,1591.466670,723.715703
1018,강남구,19.0,5,아파트,63.09,557563,2.75,3.8,72.0,78.2,...,81.2,NaN,37.485309,127.034404,38600.00,30000,2011-01-01,107.550575,1691.882920,270.721735
1020,강남구,9.0,5,아파트,67.50,557563,2.75,3.8,72.0,78.2,...,81.2,NaN,37.485309,127.034404,38600.00,31000,2011-01-01,107.550575,1691.882920,270.721735
1055,강남구,5.0,6,아파트,129.18,557563,2.75,3.8,72.0,78.2,...,81.2,NaN,37.506861,127.028750,61500.00,40000,2011-01-01,273.072763,1134.433254,285.120130
1058,강남구,1.0,6,아파트,59.61,557563,2.75,3.8,72.0,78.2,...,81.2,NaN,37.498777,127.049879,85672.73,30000,2011-01-01,156.475666,2614.395870,244.731907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125397,중랑구,10.0,27,아파트,49.77,384272,NaN,NaN,NaN,NaN,...,NaN,NaN,37.615262,127.091014,39000.00,30000,2023-09-01,531.732619,1429.809931,121.712855
1125400,중랑구,5.0,27,아파트,39.84,384272,NaN,NaN,NaN,NaN,...,NaN,NaN,37.615262,127.091014,39000.00,24000,2023-09-01,531.732619,1429.809931,121.712855
1125416,중랑구,3.0,26,아파트,50.37,384272,NaN,NaN,NaN,NaN,...,NaN,NaN,37.597149,127.081907,54000.00,28800,2023-09-01,255.627811,1952.815142,188.219471
1125346,중랑구,10.0,5,연립다세대,24.93,384272,NaN,NaN,NaN,NaN,...,NaN,NaN,37.594435,127.079118,21000.00,18000,2023-09-01,296.362951,1574.826226,495.776743


# IR = 3.5 since 23.02

In [29]:
import pandas as pd

new_data['IR'].fillna(3.5, inplace=True)

new_data

,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,LC_index,CA_index,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,JS_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School
315,강남구,4.0,8,아파트,95.73,557563,2.75,3.8,72.0,78.2,...,81.2,NaN,37.502593,127.031140,54500.00,32000,2011-01-01,458.765733,1591.466670,723.715703
1018,강남구,19.0,5,아파트,63.09,557563,2.75,3.8,72.0,78.2,...,81.2,NaN,37.485309,127.034404,38600.00,30000,2011-01-01,107.550575,1691.882920,270.721735
1020,강남구,9.0,5,아파트,67.50,557563,2.75,3.8,72.0,78.2,...,81.2,NaN,37.485309,127.034404,38600.00,31000,2011-01-01,107.550575,1691.882920,270.721735
1055,강남구,5.0,6,아파트,129.18,557563,2.75,3.8,72.0,78.2,...,81.2,NaN,37.506861,127.028750,61500.00,40000,2011-01-01,273.072763,1134.433254,285.120130
1058,강남구,1.0,6,아파트,59.61,557563,2.75,3.8,72.0,78.2,...,81.2,NaN,37.498777,127.049879,85672.73,30000,2011-01-01,156.475666,2614.395870,244.731907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125397,중랑구,10.0,27,아파트,49.77,384272,3.50,NaN,NaN,NaN,...,NaN,NaN,37.615262,127.091014,39000.00,30000,2023-09-01,531.732619,1429.809931,121.712855
1125400,중랑구,5.0,27,아파트,39.84,384272,3.50,NaN,NaN,NaN,...,NaN,NaN,37.615262,127.091014,39000.00,24000,2023-09-01,531.732619,1429.809931,121.712855
1125416,중랑구,3.0,26,아파트,50.37,384272,3.50,NaN,NaN,NaN,...,NaN,NaN,37.597149,127.081907,54000.00,28800,2023-09-01,255.627811,1952.815142,188.219471
1125346,중랑구,10.0,5,연립다세대,24.93,384272,3.50,NaN,NaN,NaN,...,NaN,NaN,37.594435,127.079118,21000.00,18000,2023-09-01,296.362951,1574.826226,495.776743


# Impute Crime_Rates NaNs with region-wise mean values

In [30]:
# Calculate Region_Name-wise mean values
cr_mean = new_data.groupby('Region_Name')['Crime_Rates'].transform('mean')

# Fill missing values with the corresponding Region_Name-wise mean
new_data['Crime_Rates'].fillna(cr_mean, inplace=True)

# Print info and display the updated DataFrame
new_data.info()
new_data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1189434 entries, 315 to 1125360
Data columns (total 22 columns):
 #   Column                           Non-Null Count    Dtype         
---  ------                           --------------    -----         
 0   Region_Name                      1189434 non-null  object        
 1   Floor                            1189434 non-null  float64       
 2   Building_Age                     1189434 non-null  int64         
 3   Building_Use                     1189434 non-null  object        
 4   JS_BA                            1189434 non-null  float64       
 5   Population                       1189434 non-null  int64         
 6   IR                               1189434 non-null  float64       
 7   UR                               1182725 non-null  float64       
 8   LC_index                         1182725 non-null  float64       
 9   CA_index                         1182725 non-null  float64       
 10  TC_index                    

,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,LC_index,CA_index,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,JS_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School
315,강남구,4.0,8,아파트,95.73,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.502593,127.031140,54500.00,32000,2011-01-01,458.765733,1591.466670,723.715703
1018,강남구,19.0,5,아파트,63.09,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.485309,127.034404,38600.00,30000,2011-01-01,107.550575,1691.882920,270.721735
1020,강남구,9.0,5,아파트,67.50,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.485309,127.034404,38600.00,31000,2011-01-01,107.550575,1691.882920,270.721735
1055,강남구,5.0,6,아파트,129.18,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.506861,127.028750,61500.00,40000,2011-01-01,273.072763,1134.433254,285.120130
1058,강남구,1.0,6,아파트,59.61,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.498777,127.049879,85672.73,30000,2011-01-01,156.475666,2614.395870,244.731907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125397,중랑구,10.0,27,아파트,49.77,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.615262,127.091014,39000.00,30000,2023-09-01,531.732619,1429.809931,121.712855
1125400,중랑구,5.0,27,아파트,39.84,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.615262,127.091014,39000.00,24000,2023-09-01,531.732619,1429.809931,121.712855
1125416,중랑구,3.0,26,아파트,50.37,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.597149,127.081907,54000.00,28800,2023-09-01,255.627811,1952.815142,188.219471
1125346,중랑구,10.0,5,연립다세대,24.93,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.594435,127.079118,21000.00,18000,2023-09-01,296.362951,1574.826226,495.776743


In [31]:
nans = new_data.isnull().sum()
nans

Region_Name                             0
Floor                                   0
Building_Age                            0
Building_Use                            0
JS_BA                                   0
Population                              0
IR                                      0
UR                                   6709
LC_index                             6709
CA_index                             6709
TC_index                             6709
SDT_index                          148446
HSP_index                           23233
Crime_Rates                             0
latitude                                0
longitude                               0
Sell_Price                              0
JS_Price                                0
YearMonth                               0
Shortest_Distance_to_Subway           982
Shortest_Distance_to_University       982
Shortest_Distance_to_School           982
dtype: int64

In [32]:
new_data

,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,LC_index,CA_index,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,JS_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School
315,강남구,4.0,8,아파트,95.73,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.502593,127.031140,54500.00,32000,2011-01-01,458.765733,1591.466670,723.715703
1018,강남구,19.0,5,아파트,63.09,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.485309,127.034404,38600.00,30000,2011-01-01,107.550575,1691.882920,270.721735
1020,강남구,9.0,5,아파트,67.50,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.485309,127.034404,38600.00,31000,2011-01-01,107.550575,1691.882920,270.721735
1055,강남구,5.0,6,아파트,129.18,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.506861,127.028750,61500.00,40000,2011-01-01,273.072763,1134.433254,285.120130
1058,강남구,1.0,6,아파트,59.61,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.498777,127.049879,85672.73,30000,2011-01-01,156.475666,2614.395870,244.731907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125397,중랑구,10.0,27,아파트,49.77,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.615262,127.091014,39000.00,30000,2023-09-01,531.732619,1429.809931,121.712855
1125400,중랑구,5.0,27,아파트,39.84,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.615262,127.091014,39000.00,24000,2023-09-01,531.732619,1429.809931,121.712855
1125416,중랑구,3.0,26,아파트,50.37,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.597149,127.081907,54000.00,28800,2023-09-01,255.627811,1952.815142,188.219471
1125346,중랑구,10.0,5,연립다세대,24.93,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.594435,127.079118,21000.00,18000,2023-09-01,296.362951,1574.826226,495.776743


In [60]:
data_APT = new_data[new_data['Building_Use'].str.contains('아파트')]
data_APT

,Year,Month,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,JS_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School
315,2011,1,강남구,4.0,8,아파트,95.73,557563,2.75,3.8,...,81.2,NaN,37.502593,127.031140,54500.00,32000,201101,458.765733,1591.466670,723.715703
1018,2011,1,강남구,19.0,5,아파트,63.09,557563,2.75,3.8,...,81.2,NaN,37.485309,127.034404,38600.00,30000,201101,107.550575,1691.882920,270.721735
1020,2011,1,강남구,9.0,5,아파트,67.50,557563,2.75,3.8,...,81.2,NaN,37.485309,127.034404,38600.00,31000,201101,107.550575,1691.882920,270.721735
1055,2011,1,강남구,5.0,6,아파트,129.18,557563,2.75,3.8,...,81.2,NaN,37.506861,127.028750,61500.00,40000,201101,273.072763,1134.433254,285.120130
1058,2011,1,강남구,1.0,6,아파트,59.61,557563,2.75,3.8,...,81.2,NaN,37.498777,127.049879,85672.73,30000,201101,156.475666,2614.395870,244.731907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125358,2023,9,중랑구,1.0,27,아파트,49.77,384272,NaN,NaN,...,NaN,NaN,37.615262,127.091014,39000.00,28500,202309,531.732619,1429.809931,121.712855
1125372,2023,9,중랑구,13.0,26,아파트,59.76,384272,NaN,NaN,...,NaN,NaN,37.597149,127.081907,54000.00,31500,202309,255.627811,1952.815142,188.219471
1125397,2023,9,중랑구,10.0,27,아파트,49.77,384272,NaN,NaN,...,NaN,NaN,37.615262,127.091014,39000.00,30000,202309,531.732619,1429.809931,121.712855
1125400,2023,9,중랑구,5.0,27,아파트,39.84,384272,NaN,NaN,...,NaN,NaN,37.615262,127.091014,39000.00,24000,202309,531.732619,1429.809931,121.712855


In [34]:
data_officetel = new_data[new_data['Building_Use'].str.contains('오피스텔')]
data_officetel

,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,LC_index,CA_index,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,JS_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School
6454,강남구,10.0,20,오피스텔,22.77,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.523582,127.045128,9250.00,7000,2011-01-01,625.868025,1450.060983,405.257077
48235,강남구,7.0,8,오피스텔,40.56,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.518459,127.026052,24400.00,13000,2011-01-01,730.847519,469.699738,548.727230
57243,강남구,15.0,6,오피스텔,57.27,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.493660,127.031016,33733.33,22000,2011-01-01,452.912990,1360.509083,154.541041
57695,강남구,7.0,7,오피스텔,35.34,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.494073,127.031311,19500.00,14000,2011-01-01,431.552415,1398.985052,155.616332
2307,강동구,4.0,4,오피스텔,56.04,486546,2.75,3.8,72.0,78.2,...,81.2,0.947923,37.537012,127.130419,19450.00,14000,2011-01-01,855.278859,1933.788220,329.033365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125366,마포구,8.0,17,오피스텔,30.06,365111,3.50,NaN,NaN,NaN,...,NaN,1.268818,37.551994,126.956516,19000.00,17000,2023-09-01,278.166797,946.585532,381.300209
1125505,성동구,9.0,3,오피스텔,18.63,279374,3.50,NaN,NaN,NaN,...,NaN,0.899977,37.563143,127.032841,24900.00,21400,2023-09-01,505.463661,1416.396158,285.447552
1125391,송파구,5.0,12,오피스텔,39.49,657101,3.50,NaN,NaN,NaN,...,NaN,0.897009,37.495723,127.122150,25300.00,23000,2023-09-01,251.077067,2769.771994,455.610522
1125371,영등포구,7.0,2,오피스텔,29.41,376139,3.50,NaN,NaN,NaN,...,NaN,1.450147,37.537253,126.891957,21000.00,21500,2023-09-01,220.046666,3189.431156,451.910210


In [35]:
data_townhouse = new_data[new_data['Building_Use'].str.contains('연립다세대')]
data_townhouse

,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,LC_index,CA_index,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,JS_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School
340,강남구,4.0,10,연립다세대,64.64,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.495968,127.043258,35000.0,23000,2011-01-01,816.989380,2469.693605,411.014080
9341,강동구,4.0,1,연립다세대,45.63,486546,2.75,3.8,72.0,78.2,...,81.2,0.947923,37.551542,127.138233,24500.0,17000,2011-01-01,957.106297,3034.365920,357.122247
17729,강동구,1.0,17,연립다세대,49.46,486546,2.75,3.8,72.0,78.2,...,81.2,0.947923,37.536155,127.140008,19950.0,12000,2011-01-01,648.569915,2002.767428,497.931692
35182,강동구,2.0,25,연립다세대,79.47,486546,2.75,3.8,72.0,78.2,...,81.2,0.947923,37.549889,127.156945,70000.0,22500,2011-01-01,547.861257,4069.883399,99.191064
35183,강동구,1.0,25,연립다세대,122.61,486546,2.75,3.8,72.0,78.2,...,81.2,0.947923,37.549889,127.156945,70000.0,29000,2011-01-01,547.861257,4069.883399,99.191064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125313,양천구,4.0,7,연립다세대,29.85,438353,3.50,NaN,NaN,NaN,...,NaN,0.799288,37.542412,126.869706,24250.0,20200,2023-09-01,707.596340,1470.697291,468.822685
1125269,용산구,4.0,0,연립다세대,29.78,217288,3.50,NaN,NaN,NaN,...,NaN,1.418952,37.533164,126.954626,52000.0,20000,2023-09-01,803.757406,1242.808277,162.978319
1125350,은평구,5.0,8,연립다세대,34.79,463298,3.50,NaN,NaN,NaN,...,NaN,0.815561,37.625645,126.920566,29400.0,20000,2023-09-01,346.127598,2848.928516,336.668598
1125346,중랑구,10.0,5,연립다세대,24.93,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.594435,127.079118,21000.0,18000,2023-09-01,296.362951,1574.826226,495.776743


# Other Columns having NaNs are handled with XGBoost

In [50]:
# Define your features and target variable
X = data_APT.drop(columns=['JS_Price'])
y = data_APT['JS_Price']

# Identify categorical columns that may benefit from Target Encoding
categorical_columns = ['Region_Name', 'Building_Use', 'YearMonth']

# Apply Target Encoding to the selected categorical columns
encoder = TargetEncoder(cols=categorical_columns)
X_encoded = encoder.fit_transform(X, y)




In [51]:
columns_to_merge = ['JS_Price']
new_data= new_data[columns_to_merge]

# Merge the extracted columns with the other dataset using a common key (e.g., an index or a shared column)
# For example, if both datasets have an index, you can merge them as follows:
merged_data = pd.merge(X_encoded, new_data, left_index=True, right_index=True)
merged_data

,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,LC_index,CA_index,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School,JS_Price
315,57630.344239,4.0,8,39130.905834,95.73,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.502593,127.031140,54500.00,25148.189625,458.765733,1591.466670,723.715703,32000
1018,57630.344239,19.0,5,39130.905834,63.09,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.485309,127.034404,38600.00,25148.189625,107.550575,1691.882920,270.721735,30000
1020,57630.344239,9.0,5,39130.905834,67.50,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.485309,127.034404,38600.00,25148.189625,107.550575,1691.882920,270.721735,31000
1055,57630.344239,5.0,6,39130.905834,129.18,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.506861,127.028750,61500.00,25148.189625,273.072763,1134.433254,285.120130,40000
1058,57630.344239,1.0,6,39130.905834,59.61,557563,2.75,3.8,72.0,78.2,...,81.2,1.386371,37.498777,127.049879,85672.73,25148.189625,156.475666,2614.395870,244.731907,30000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125358,25345.422036,1.0,27,39130.905834,49.77,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.615262,127.091014,39000.00,53689.792208,531.732619,1429.809931,121.712855,28500
1125372,25345.422036,13.0,26,39130.905834,59.76,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.597149,127.081907,54000.00,53689.792208,255.627811,1952.815142,188.219471,31500
1125397,25345.422036,10.0,27,39130.905834,49.77,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.615262,127.091014,39000.00,53689.792208,531.732619,1429.809931,121.712855,30000
1125400,25345.422036,5.0,27,39130.905834,39.84,384272,3.50,NaN,NaN,NaN,...,NaN,1.086769,37.615262,127.091014,39000.00,53689.792208,531.732619,1429.809931,121.712855,24000


In [52]:
import xgboost as xgb
import pandas as pd
import numpy as np

# Load your dataset (replace 'your_dataset.csv' with the actual file path)
data_APT = merged_data

# Columns with missing values
columns_with_missing = ['LC_index', 'CA_index', 'TC_index', 'SDT_index', 'Shortest_Distance_to_Subway',
                        'Shortest_Distance_to_University', 'Shortest_Distance_to_School', 'HSP_index', 'UR']

# Split into training and imputation sets based on missing values in the target columns
train_set = data_APT.dropna(subset=columns_with_missing)
impute_set = data_APT[data_APT[columns_with_missing].isnull().any(axis=1)]

# Separate features and target variables
X_train = train_set.drop(columns=columns_with_missing)
X_impute = impute_set.drop(columns=columns_with_missing)

# Initialize an empty dictionary to store models
models = {}

# Set parameters for XGBoost model (you can adjust these based on your requirements)
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100  # Use 'n_estimators' instead of 'num_boost_round'
}

# Train separate XGBoost models for each target column
for column in columns_with_missing:
    y_train = train_set[column]
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    models[column] = model

# Predict missing values in the imputation set for each target column
imputed_values = pd.DataFrame()
for column in columns_with_missing:
    model = models[column]
    imputed_values[column] = model.predict(X_impute)

# Replace missing values in the original dataset with the predicted values
data_APT.loc[data_APT[columns_with_missing].isnull().any(axis=1), columns_with_missing] = imputed_values.values

# The missing values have been imputed using XGBoost
nans = data_APT.isnull().sum()
print(nans)
data_APT.info()
data_APT


Region_Name                        0
Floor                              0
Building_Age                       0
Building_Use                       0
JS_BA                              0
Population                         0
IR                                 0
UR                                 0
LC_index                           0
CA_index                           0
TC_index                           0
SDT_index                          0
HSP_index                          0
Crime_Rates                        0
latitude                           0
longitude                          0
Sell_Price                         0
YearMonth                          0
Shortest_Distance_to_Subway        0
Shortest_Distance_to_University    0
Shortest_Distance_to_School        0
JS_Price                           0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1053327 entries, 315 to 1125416
Data columns (total 22 columns):
 #   Column                           Non-Null Count    Dt

,Region_Name,Floor,Building_Age,Building_Use,JS_BA,Population,IR,UR,LC_index,CA_index,...,HSP_index,Crime_Rates,latitude,longitude,Sell_Price,YearMonth,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School,JS_Price
315,57630.344239,4.0,8,39130.905834,95.73,557563,2.75,3.371603,75.893532,82.497063,...,74.134789,1.386371,37.502593,127.031140,54500.00,25148.189625,445.785858,2058.513672,320.359161,32000
1018,57630.344239,19.0,5,39130.905834,63.09,557563,2.75,3.371603,75.893433,82.497063,...,74.134857,1.386371,37.485309,127.034404,38600.00,25148.189625,565.756897,1863.666992,297.848663,30000
1020,57630.344239,9.0,5,39130.905834,67.50,557563,2.75,3.371603,75.893394,82.497063,...,74.134117,1.386371,37.485309,127.034404,38600.00,25148.189625,559.690918,1863.666992,297.848663,31000
1055,57630.344239,5.0,6,39130.905834,129.18,557563,2.75,3.371603,75.893532,82.497063,...,74.134926,1.386371,37.506861,127.028750,61500.00,25148.189625,417.186707,1523.690308,377.802460,40000
1058,57630.344239,1.0,6,39130.905834,59.61,557563,2.75,3.371603,75.894302,82.496399,...,74.100250,1.386371,37.498777,127.049879,85672.73,25148.189625,498.747833,2566.429199,282.421722,30000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125358,25345.422036,1.0,27,39130.905834,49.77,384272,3.50,2.699322,109.782448,110.263451,...,152.750809,1.086769,37.615262,127.091014,39000.00,53689.792208,490.444672,1446.200562,174.239624,28500
1125372,25345.422036,13.0,26,39130.905834,59.76,384272,3.50,2.699322,109.783394,110.265808,...,152.761292,1.086769,37.597149,127.081907,54000.00,53689.792208,395.779327,1707.468994,247.954987,31500
1125397,25345.422036,10.0,27,39130.905834,49.77,384272,3.50,2.699322,109.782448,110.263451,...,152.750809,1.086769,37.615262,127.091014,39000.00,53689.792208,490.444672,1446.200562,174.239624,30000
1125400,25345.422036,5.0,27,39130.905834,39.84,384272,3.50,2.699322,109.782448,110.263451,...,152.750809,1.086769,37.615262,127.091014,39000.00,53689.792208,513.368286,1446.200562,172.365936,24000


In [53]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import mutual_info_regression

# Assuming 'new_data' is your DataFrame
X = data_APT.drop(columns=['JS_Price'])  # Features
y = data_APT['JS_Price']  # Target

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# 1. SHAP with XGBoost
# Train an XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=283)
xgb_model.fit(X_normalized, y)

# Create a SHAP explainer for the trained XGBoost model
explainer_shap = shap.Explainer(xgb_model)
shap_values = explainer_shap.shap_values(X_normalized)

# Compute absolute SHAP values for feature ranking
shap_scores_abs = np.abs(shap_values).mean(axis=0)
shap_ranking = pd.Series(shap_scores_abs, index=X.columns).sort_values(ascending=False)

# 2. Permutation Importance
# Compute permutation importance
result_permutation = permutation_importance(xgb_model, X_normalized, y, n_repeats=10, random_state=42)
permutation_importance_scores = result_permutation.importances_mean
permutation_ranking = pd.Series(permutation_importance_scores, index=X.columns).sort_values(ascending=False)

# 3. Mutual Information
# Compute mutual information scores for selected features
mi_scores = mutual_info_regression(X_normalized, y)
mi_ranking = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

# 4. Random Forest
# Train a Random Forest model on all features
rf_model_rf = RandomForestRegressor(n_estimators=100, random_state=293)
rf_model_rf.fit(X_normalized, y)

# Compute feature importances from Random Forest for all features
rf_feature_importances = pd.Series(rf_model_rf.feature_importances_, index=X.columns).sort_values(ascending=False)

# Now you have rankings and feature importances for each method


# APT Results

In [54]:



# Define a function for min-max scaling
min_max_scaler_mi_linear_rf_shap_permutation = lambda x: (x - x.min()) / (x.max() - x.min())

# Normalize feature importance rankings for mutual information (MI), linear regression, Random Forest (RF), SHAP, and permutation importance
normalized_mi_ranking = min_max_scaler_mi_linear_rf_shap_permutation(mi_ranking)
normalized_rf_ranking = min_max_scaler_mi_linear_rf_shap_permutation(rf_feature_importances)
normalized_shap_ranking = min_max_scaler_mi_linear_rf_shap_permutation(shap_ranking)
normalized_permutation_ranking = min_max_scaler_mi_linear_rf_shap_permutation(permutation_ranking)

# Define weights for each feature importance metric
weight_mi = 0.25
weight_rf = 0.25
weight_shap = 0.25
weight_permutation = 0.25

# Combine normalized rankings using weighted averages
combined_ranking = (
    weight_mi * normalized_mi_ranking +
    weight_rf * normalized_rf_ranking +
    weight_shap * normalized_shap_ranking +
    weight_permutation * normalized_permutation_ranking
)

# Create a DataFrame with feature names and combined rankings
combined_ranking_df = pd.DataFrame({
    'Feature': combined_ranking.index,
    'Combined_Ranking': combined_ranking.values
})

# Sort the features by combined ranking in descending order
sorted_features_combined = combined_ranking_df.sort_values(by='Combined_Ranking', ascending=False)

# Store the sorted features and their rankings in Rankings_Features
Rankings_Features = sorted_features_combined

# Display the feature rankings
Rankings_Features


,Feature,Combined_Ranking
12,Sell_Price,0.917844
7,JS_BA,0.726669
0,Building_Age,0.303011
19,latitude,0.299427
20,longitude,0.278701
14,Shortest_Distance_to_Subway,0.273083
13,Shortest_Distance_to_School,0.269930
15,Shortest_Distance_to_University,0.269220
8,LC_index,0.203876
10,Region_Name,0.184569


# Officetel Results

In [49]:



# Define a function for min-max scaling
min_max_scaler_mi_linear_rf_shap_permutation = lambda x: (x - x.min()) / (x.max() - x.min())

# Normalize feature importance rankings for mutual information (MI), linear regression, Random Forest (RF), SHAP, and permutation importance
normalized_mi_ranking = min_max_scaler_mi_linear_rf_shap_permutation(mi_ranking)
normalized_rf_ranking = min_max_scaler_mi_linear_rf_shap_permutation(rf_feature_importances)
normalized_shap_ranking = min_max_scaler_mi_linear_rf_shap_permutation(shap_ranking)
normalized_permutation_ranking = min_max_scaler_mi_linear_rf_shap_permutation(permutation_ranking)

# Define weights for each feature importance metric
weight_mi = 0.25
weight_rf = 0.25
weight_shap = 0.25
weight_permutation = 0.25

# Combine normalized rankings using weighted averages
combined_ranking = (
    weight_mi * normalized_mi_ranking +
    weight_rf * normalized_rf_ranking +
    weight_shap * normalized_shap_ranking +
    weight_permutation * normalized_permutation_ranking
)

# Create a DataFrame with feature names and combined rankings
combined_ranking_df = pd.DataFrame({
    'Feature': combined_ranking.index,
    'Combined_Ranking': combined_ranking.values
})

# Sort the features by combined ranking in descending order
sorted_features_combined = combined_ranking_df.sort_values(by='Combined_Ranking', ascending=False)

# Store the sorted features and their rankings in Rankings_Features
Rankings_Features = sorted_features_combined

# Display the feature rankings
Rankings_Features


,Feature,Combined_Ranking
7,JS_BA,0.782098
12,Sell_Price,0.714747
20,longitude,0.305806
19,latitude,0.284241
15,Shortest_Distance_to_University,0.272354
13,Shortest_Distance_to_School,0.249320
14,Shortest_Distance_to_Subway,0.249069
5,HSP_index,0.203136
0,Building_Age,0.156145
9,Population,0.150220


# Townhouse Results

In [43]:



# Define a function for min-max scaling
min_max_scaler_mi_linear_rf_shap_permutation = lambda x: (x - x.min()) / (x.max() - x.min())

# Normalize feature importance rankings for mutual information (MI), linear regression, Random Forest (RF), SHAP, and permutation importance
normalized_mi_ranking = min_max_scaler_mi_linear_rf_shap_permutation(mi_ranking)
normalized_rf_ranking = min_max_scaler_mi_linear_rf_shap_permutation(rf_feature_importances)
normalized_shap_ranking = min_max_scaler_mi_linear_rf_shap_permutation(shap_ranking)
normalized_permutation_ranking = min_max_scaler_mi_linear_rf_shap_permutation(permutation_ranking)

# Define weights for each feature importance metric
weight_mi = 0.25
weight_rf = 0.25
weight_shap = 0.25
weight_permutation = 0.25

# Combine normalized rankings using weighted averages
combined_ranking = (
    weight_mi * normalized_mi_ranking +
    weight_rf * normalized_rf_ranking +
    weight_shap * normalized_shap_ranking +
    weight_permutation * normalized_permutation_ranking
)

# Create a DataFrame with feature names and combined rankings
combined_ranking_df = pd.DataFrame({
    'Feature': combined_ranking.index,
    'Combined_Ranking': combined_ranking.values
})

# Sort the features by combined ranking in descending order
sorted_features_combined = combined_ranking_df.sort_values(by='Combined_Ranking', ascending=False)

# Store the sorted features and their rankings in Rankings_Features
Rankings_Features = sorted_features_combined

# Display the feature rankings
Rankings_Features


,Feature,Combined_Ranking
12,Sell_Price,0.933217
0,Building_Age,0.622963
7,JS_BA,0.613293
19,latitude,0.274470
20,longitude,0.239553
10,Region_Name,0.236939
15,Shortest_Distance_to_University,0.195008
8,LC_index,0.193562
14,Shortest_Distance_to_Subway,0.186830
13,Shortest_Distance_to_School,0.176739


# All Data Results

In [20]:



# Define a function for min-max scaling
min_max_scaler_mi_linear_rf_shap_permutation = lambda x: (x - x.min()) / (x.max() - x.min())

# Normalize feature importance rankings for mutual information (MI), linear regression, Random Forest (RF), SHAP, and permutation importance
normalized_mi_ranking = min_max_scaler_mi_linear_rf_shap_permutation(mi_ranking)
normalized_rf_ranking = min_max_scaler_mi_linear_rf_shap_permutation(rf_feature_importances)
normalized_shap_ranking = min_max_scaler_mi_linear_rf_shap_permutation(shap_ranking)
normalized_permutation_ranking = min_max_scaler_mi_linear_rf_shap_permutation(permutation_ranking)

# Define weights for each feature importance metric
weight_mi = 0.25
weight_rf = 0.25
weight_shap = 0.25
weight_permutation = 0.25

# Combine normalized rankings using weighted averages
combined_ranking = (
    weight_mi * normalized_mi_ranking +
    weight_rf * normalized_rf_ranking +
    weight_shap * normalized_shap_ranking +
    weight_permutation * normalized_permutation_ranking
)

# Create a DataFrame with feature names and combined rankings
combined_ranking_df = pd.DataFrame({
    'Feature': combined_ranking.index,
    'Combined_Ranking': combined_ranking.values
})

# Sort the features by combined ranking in descending order
sorted_features_combined = combined_ranking_df.sort_values(by='Combined_Ranking', ascending=False)

# Store the sorted features and their rankings in Rankings_Features
Rankings_Features = sorted_features_combined

# Display the feature rankings
Rankings_Features


,Feature,Combined_Ranking
12,Sell_Price,0.986482
7,JS_BA,0.684255
0,Building_Age,0.251346
19,latitude,0.189964
20,longitude,0.181056
10,Region_Name,0.145718
9,Population,0.136831
15,Shortest_Distance_to_University,0.124809
13,Shortest_Distance_to_School,0.123984
14,Shortest_Distance_to_Subway,0.121152
